In [143]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import pandas as pd
from selenium.webdriver.chrome.options import Options
import traceback
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




url="https://bcs-express.ru/category/torgovye-rekomendacii"

In [43]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response=requests.get(url, headers=headers)

soup=BeautifulSoup(response.text,"html.parser")

print(soup.find_all("div", class_=["TjB6 KSLV Ncpb E6j8","TjB6 KSLV Ncpb E6j8 IhFN"])[0].find("a", class_="iKzE").text) #title
print(soup.find_all("div", class_=["TjB6 KSLV Ncpb E6j8","TjB6 KSLV Ncpb E6j8 IhFN"])[0].find("a", class_="iKzE").attrs['href']) #link


AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [206]:
driver = webdriver.Safari()

driver.get(url)

button = driver.find_element(By.CLASS_NAME,"nlJ0")
button.click()
time.sleep(3)
columns=['title', 'description', 'publishedAt']
df=pd.DataFrame(columns=columns)
k=-1

try:
    while True:
        # Прокручиваем вниз
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollBy(0, -400);")

        time.sleep(4)
        # Ждем загрузки новых данных
        soup = BeautifulSoup(driver.page_source,"html.parser")
        page = soup.find_all("div", class_=["TjB6 KSLV Ncpb E6j8","TjB6 KSLV Ncpb E6j8 IhFN"])
        print(len(page))
        for i in range(k+1,len(page)):
            title=page[i].find("a", class_="iKzE").text
            link=page[i].find("a", class_="iKzE").attrs['href']
            main_window=driver.current_window_handle
            driver.execute_script(f"window.open('{link}', '_blank');")
            time.sleep(1)
            # Переключение на новую вкладку
            # Получаем все открытые окна
            windows = driver.window_handles
            # Переключаемся на новое окно
            for window in windows:
                if window != main_window:
                    driver.switch_to.window(window)
                    break
            try:
                WebDriverWait(driver,30).until(lambda d: d.execute_script('return document.readyState') == 'complete')
            except Exception as e:
                print("не удалось загрузить страницу; {link}")
                break
            try:
                WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.TAG_NAME,'time')))
            except Exception as e:
                print(f"ошибка с тайм {e}")
            description=' '.join(list(map(lambda element: element.text,driver.find_element(By.TAG_NAME, 'body').find_elements(By.TAG_NAME,"p")[:-8])))
            publish_date=driver.find_element(By.TAG_NAME, 'time').text
            df.loc[len(df)]=[title,description,publish_date]
            for window in windows:
                if window != main_window:
                    driver.switch_to.window(window)
                    break
            driver.close()
            driver.switch_to.window(main_window)
            k=i
    
        if len(df)>=200:
              break
            
finally:
      driver.quit()


36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
ошибка с тайм Message: 

126
132
138
144
150
156
162
168
174
180
186
192
198
204


<function selenium.webdriver.support.expected_conditions.presence_of_element_located(locator: Tuple[str, str]) -> Callable[[Union[selenium.webdriver.remote.webdriver.WebDriver, selenium.webdriver.remote.webelement.WebElement]], selenium.webdriver.remote.webelement.WebElement]>

In [114]:
try:
    driver=webdriver.Safari()
    driver.get("https://bcs-express.ru/novosti-i-analitika/ideia-v-aktsiiakh-vtb-potentsial-naia-dokhodnost-10-k-kontsu-iiunia")
    description=driver.find_element(By.TAG_NAME, 'time').text
    print(description)
finally:
    driver.quit()

13 мая в 15:37
